In [ ]:
# THE STELLAR INTERPRETATION OF QUANTUM MECHANICS
# THANKS TO ETTORE MAJORANA. VERSION 3.0
#
# Now with density matrices. See settings below.
#
# Choose:
n=4
# Choose n where n is the dimensionality of the randomly 
# generated quantum system, leading to n constellations of
# n-1 stars in the sky, each constellation
# corresponding to an eigenvector of the density matrix.
# The "height" or "size" of the stars in each constellation
# is determined by the eigenvalue squared, and the color 
# of the stars within a constellation are all the same,
# randomly chosen.

import vpython
import numpy as np
import scipy.linalg
import math
import mpmath
import qutip
import random

# Stereographically projects an XYZ cartesian coordinate on 
# the Riemann sphere to a point on the complex plane.
def flatten(x, y, z):
    if z == 1:
        return Inf 
    else:
        return complex(x/(1-z), y/(1-z))

# Reverse stereographically projects a point on the complex plane
# to an XYZ cartesian coordinate on the Riemann sphere.
def rollup(z):
    x = z.real
    y = z.imag
    return (2*x)/(1+(x**2)+(y**2)), \
            (2*y)/(1+(x**2)+(y**2)), \
            (-1+(x**2)+(y**2))/(1+(x**2)+(y**2))
        
# Takes a complex vector, interprets its ordered components as
# the coefficients of a polynomial in one variable, and solves for
# the roots, giving an unordered set of complex numbers. 
# These complex points are reverse stereographically projected onto 
# the Riemann sphere, and the XYZ cartesian coordinates of these points 
# are returned. When there's a ZeroDivisionError aka we're at infinity,
# we map to the point (0,0,1) aka the North Pole.
def constellate(v):
    try:
        roots = mpmath.polyroots(v)
    except:
         return [[0,0,1] for i in range(len(v))]
    return [[float(a) for a in rollup(root)] for root in roots]

# Takes a matrix, in this case, a density matrix, and finds its spectrum.
# Two lists are returned: of eigenvalues and the corresponding constellations of each eigenvector,
# signifying the constellations at what "height" or "scale" between heaven and earth.
def emanate(m):
    l, v = np.linalg.eig(m)
    return l, [constellate(v[i]) for i in range(len(v))]

# Returns a randomly generated quantum system:
# a random density matrix for the state,
# a random hermitian matrix for the hamiltonian,
# aka the energy operator aka the time evolver.
def bang(n):
    return qutip.rand_dm(n).full(), \
        qutip.rand_herm(n).full()

# Initialize quantum simulation
state, hamiltonian = bang(n)

# Initialize video settings
vpython.scene.range = 2
# Set our initial vantage point at random.
vpython.scene.forward = vpython.vector(random.random(), random.random(), random.random())

# Create the celestial sphere, and the starting constellations at each level of heaven.
god = vpython.sphere(pos=vpython.vector(0,0,0), radius=2, color=vpython.color.blue, opacity=0.2)
heights, heavens = emanate(state)
levels = []
hue = random.random()
for i in range(len(heavens)):
    height = heights[i]
    constellation = heavens[i]
    # Each level of heaven (each eigenspace of the density matrix)
    # gets randomly assigned a color for its stars.
    color = vpython.vector(random.random(), 1, 1)
    # Stars in any constellation have radii no smaller than 0.1, but otherwise
    # their radius is proportional to the eigenvalue squared corresponding
    # to the constellated eigenvector.
    # Note a duality:
    #  We could place the levels of heaven one atop the other at distances 
    #  given by the eigenvalues squared, and have all stars across all heavens
    #  be the same size. Or we could have all stars in all constellations
    #  hang on the unit sphere, but rather make the sizes of the individual stars 
    #  within a constellation vary with the eigenvalue squared. You'll notice,
    #  in a profound way, it comes to the same thing. Here the latter representation
    #  is more perspicacious. 
    stars = [vpython.sphere(pos=2*vpython.vector(*star)*abs(height), radius=abs(height)/2.,\
                color=vpython.color.hsv_to_rgb(color), opacity=0.4, emissive=True )\
                    for star in constellation]
    levels.append(stars)

# Time evolution:
# Exponentiates the hamiltonian, integrating up to the current
# time step, giving a unitary transformation implementing
# continuous time evolution up to now. The present density
# matrix is wedged between this unitary time evolution operator
# and its conjugate transpose to obtain the future density matrix. 
# The resulting matrix is constellated,
# and the locations of the stars at each level of heaven are updated.
# NB: In a sense, the stars are constantly swapping places.
# Adjust: 
#  dt gives the width of a timestep in the simulation
#  rate(f) sets the frequency of updates in real time
#    to increase the effect of verisimilitude.
t = 0
dt = 0.002
while True:
    vpython.rate(10000)
    time_step = scipy.linalg.expm(-2*math.pi*complex(0,1)*hamiltonian*t)
    tomorrows_heights, tomorrows_heavens = emanate(time_step.dot(state).dot(time_step.conj().T))
    for i in range(len(tomorrows_heavens)):
        tomorrows_height = tomorrows_heights[i]
        tomorrows_constellation = tomorrows_heavens[i]
        stars = levels[i]
        for j in range(len(stars)):
            stars[j].pos = 2*abs(tomorrows_height)*vpython.vector(*tomorrows_constellation[j])
            stars[j].radius = float(abs(tomorrows_height))/2.
    t += dt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>